# CS 484 Final Project

Plan:
1. read some more papers and find all the datasets
  a) we can use resnet50 as encoder, and then I think we build a more complicated decoder
2. try to see if we can get an overfitted model to work
3. try to see if we can extend that model
4. Great Success


## Abstract

TODO

## Data Synthesis

In [1]:
import random
import math
import numbers
import platform
import copy

# Importing essential libraries for basic image manipulations.
import numpy as np
import PIL
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm import tqdm

# We import some of the main PyTorch and TorchVision libraries.
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF
import torchvision.models as models

## Model

In [ ]:
### model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)


## Reconstructing the image from the disparity

In [ ]:
#no idea
def reconStruct(im, dmap):
    return im

## Loss Function

source of loss function:
from paper

Difficulties with loss function:
- keeping the loss function only with tensor operations to keep the .backward() functionality valid

In [ ]:
import torch
import torch.nn.functional as F
# from pytorch_ssim import SSIM  
from ignite.metrics import SSIM

def gradient_x(img):
    return img[:, :, :, :-1] - img[:, :, :, 1:]

def gradient_y(img):
    return img[:, :, :-1, :] - img[:, :, 1:, :]


def appearance_loss(img, recon, alpha):
    l1 = torch.abs(img - recon)
    ssim_loss = SSIM(img, recon)
    return (alpha * ssim_loss.mean()) + ((1 - alpha) * l1.mean())

def disparity_smoothness(disp, img):
    disp_grad_x = gradient_x(disp)
    disp_grad_y = gradient_y(disp)
    img_grad_x = gradient_x(img)
    img_grad_y = gradient_y(img)

    weights_x = torch.exp(-torch.mean(torch.abs(img_grad_x), 1, keepdim=True))
    weights_y = torch.exp(-torch.mean(torch.abs(img_grad_y), 1, keepdim=True))

    smoothness_x = disp_grad_x * weights_x
    smoothness_y = disp_grad_y * weights_y

    return smoothness_x.abs().mean() + smoothness_y.abs().mean()

def lr_consistency_loss(disp_left, disp_right):
    """

    Args:
        disp_left:  [B, 1, H, W] disparity map from left view (predicts right image)
        disp_right: [B, 1, H, W] disparity map from right view (predicts left image)

    Returns:
        Scalar tensor representing left-right consistency loss.
    """
    B, _, H, W = disp_left.shape

    # Create normalized horizontal coordinate grid
    grid_x = torch.linspace(-1.0, 1.0, W).view(1, 1, 1, W).expand(B, 1, H, W).to(disp_left.device)
    grid_y = torch.linspace(-1.0, 1.0, H).view(1, 1, H, 1).expand(B, 1, H, W).to(disp_left.device)

    # Normalize disparity to [-1, 1] range
    disp_left_norm = disp_left / ((W - 1) / 2)
    disp_right_norm = disp_right / ((W - 1) / 2)

    # Sample right disparity using left disparity to warp
    grid_warped_right = torch.cat([grid_x - disp_left_norm, grid_y], dim=1)
    grid_warped_right = grid_warped_right.permute(0, 2, 3, 1)  # [B, H, W, 2]
    sampled_right = F.grid_sample(disp_right, grid_warped_right, mode='bilinear', padding_mode='border', align_corners=True)

    # Sample left disparity using right disparity to warp
    grid_warped_left = torch.cat([grid_x + disp_right_norm, grid_y], dim=1)
    grid_warped_left = grid_warped_left.permute(0, 2, 3, 1)
    sampled_left = F.grid_sample(disp_left, grid_warped_left, mode='bilinear', padding_mode='border', align_corners=True)

    # L1 loss between disparity and its warped counterpart
    loss_left = torch.abs(disp_left - sampled_right).mean()
    loss_right = torch.abs(disp_right - sampled_left).mean()

    return loss_left + loss_right


def custom_loss(imL, imR, dL, dR, recImL, recImR, alpha=0.85, alpha_ap=1.0, alpha_ds=0.1, alpha_lr=1.0):

    loss_ap = appearance_loss(imL, recImL, alpha) + appearance_loss(imR, recImR, alpha)

    loss_ds = disparity_smoothness(dL, imL) + disparity_smoothness(dR, imR)

    loss_lr = lr_consistency_loss(dL, dR)
    # --- Total Loss ---
    total_loss = alpha_ap * loss_ap + alpha_ds * loss_ds + alpha_lr * loss_lr
    return total_loss


In [ ]:


def SSIM(self, x, y):
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    mu_x = slim.avg_pool2d(x, 3, 1, 'VALID')
    mu_y = slim.avg_pool2d(y, 3, 1, 'VALID')

    sigma_x  = slim.avg_pool2d(x ** 2, 3, 1, 'VALID') - mu_x ** 2
    sigma_y  = slim.avg_pool2d(y ** 2, 3, 1, 'VALID') - mu_y ** 2
    sigma_xy = slim.avg_pool2d(x * y , 3, 1, 'VALID') - mu_x * mu_y

    SSIM_n = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
    SSIM_d = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)

    SSIM = SSIM_n / SSIM_d

    return tF.clip_by_value((1 - SSIM) / 2, 0, 1)
# def appMatchLoss(im, recIm):


# def custom_loss(imL, imR, dl, dr, recImL, recImR):


my_loss_function = lambda x : 1

## Validation

In [ ]:
# validate the output disparity map against the ground truth
def validateAgainstGroundTruthSingle(dl, gt):
    #idk how im going to do this yet, will need to figure out dimensions and shit
    return 0.5

def validateAgainstGroundTruth(val_loader, net):
    net = net.cuda()
    ttlDiff = 0
    with torch.no_grad():
        for i, data in enumerate(val_loader):

            imL, imR, gtDl, gtDr = data
            # if USE_GPU:
            imL, imR, gtDl, gtDr = imL.cuda(), imR.cuda(), gtDl.cuda(), gtDr.cuda()

            dL, dR = net.forward(imL)
            ttlDiff += validateAgainstGroundTruthSingle(dL, gtDl) + validateAgainstGroundTruthSingle(dR, gtDr)
    
    return ttlDiff/(2*len(val_loader))



def validate(val_loader, net, loss_function = custom_loss):

    net.eval()
    val_loss = 0

    # if USE_GPU:
    net = net.cuda()
    with torch.no_grad():
        for i, data in enumerate(val_loader):

            imL, imR, gtDl, gtDr = data
            # if USE_GPU:
            imL, imR, gtDl, gtDr = imL.cuda(), imR.cuda(), gtDl.cuda(), gtDr.cuda()


            dL, dR = net.forward(imL)
            val_loss += loss_function(imL, imR, dL, dR, reconStruct(imL, dR, True), reconStruct(imR, dL, False)).item()

    # return the values of loss and mIOU (averaged over the loaded data)
    return val_loss/len(val_loader.dataset)


## Training

In [ ]:
def train(train_loader, net, optimizer, loss_graph, loss_function = custom_loss):

    net.train()
    # if USE_GPU:
    net = net.cuda()
    val_loss = 0
    for i, data in enumerate(train_loader):

        imL, imR, gtDl, gtDr = data
        # if USE_GPU:
        imL, imR, gtDl, gtDr = imL.cuda(), imR.cuda(), gtDl.cuda(), gtDr.cuda()

        optimizer.zero_grad()
        dL, dR = net.forward(imL)
        main_loss = loss_function(imL, imR, dL, dR, reconStruct(imL, dR, True), reconStruct(imR, dL, False)).item()
        loss = main_loss.item()
        val_loss += loss
        loss_graph.append(loss)
        main_loss.backward()
        optimizer.step()
        # loss_graph.append() Populate this list to graph the loss

    # Return the value of loss (averaged over the loaded data)
    return val_loss/len(train_loader)

First, we are going to train an overfit net to a single image to get a sanity check that the error function and model are working as intended

In [ ]:
def train_net(loader, net, optimizer, EPOCH):
    net.train()

    print("Starting Training...")

    loss_graph = []

    fig = plt.figure(figsize=(12,6))
    plt.subplots_adjust(bottom=0.2,right=0.85,top=0.95)
    ax = fig.add_subplot(1,1,1)

    for e in range(EPOCH):
        loss = train(loader, net, optimizer, loss_graph)
        ax.clear()
        ax.set_xlabel('iterations')
        ax.set_ylabel('loss value')
        ax.set_title('Training loss curve for OVERFIT_NET')
        ax.plot(loss_graph, label='training loss')
        ax.legend(loc='upper right')
        fig.canvas.draw()
        print("Epoch: {} Loss: {}".format(e, loss))

In [ ]:
%%time

overfit_net = copy.deepcopy(untrained_net)

EPOCH = 40

optimizer = get_optimizer(overfit_net)

train_net(sanity_loader, overfit_net, optimizer, EPOCH)

torch.save(overfit_net.state_dict(), "overfit_net.pth")
torch.save(untrained_net.state_dict(), "untrained_net.pth")

Now for the full model

In [ ]:
%%time

trained_net = copy.deepcopy(untrained_net)

EPOCH = 10

optimizer = get_optimizer(trained_net)

train_net(train_loader, trained_net, optimizer, EPOCH)

torch.save(trained_net.state_dict(), "trained_net.pth")

## Evaluation

## Conclusion

TODO